In [ ]:
import pandas as pd
import numpy as np

### Reading Data and preparing it

In [ ]:
df = pd.read_csv('../data/athlete_events.csv')
regions = pd.read_csv('../data/noc_regions.csv')
df_21 = pd.read_csv('../data/Tokyo_2021_dataset.csv')

In [ ]:
# Merge results with the region dataset
df = pd.merge(df, regions, left_on='NOC', right_on='NOC')
df = df.query('Season == "Summer"') # only interested in summer Olympics in this project

In [ ]:
# Replacing the country name with common known names
df.replace('USA', "United States of America", inplace = True)
df.replace('Tanzania', "United Republic of Tanzania", inplace = True)
df.replace('Democratic Republic of Congo', "Democratic Republic of the Congo", inplace = True)
df.replace('Congo', "Republic of the Congo", inplace = True)
df.replace('Lao', "Laos", inplace = True)
df.replace('Syrian Arab Republic', "Syria", inplace = True)
df.replace('Serbia', "Republic of Serbia", inplace = True)
df.replace('Czechia', "Czech Republic", inplace = True)
df.replace('UAE', "United Arab Emirates", inplace = True)
df.replace('UK', "United Kingdom", inplace = True)
df_21.replace('Great Britain', "United Kingdom", inplace = True)
df_21.replace("People's Republic of China", "China", inplace = True)
df_21.replace("ROC", "Russia", inplace = True)

In [ ]:
def host_country(col):
    if col == "Rio de Janeiro":
        return "Brazil"
    elif col == "London":
        return "United Kingdom"
    elif col == "Beijing":
        return  "China"
    elif col == "Athina":
        return  "Greece"
    elif col == "Sydney" or col == "Melbourne":
        return  "Australia"
    elif col == "Atlanta" or col == "Los Angeles" or col == "St. Louis":
        return  "United States of America"
    elif col == "Barcelona":
        return  "Spain"
    elif col == "Seoul":
        return  "South Korea"
    elif col == "Moskva":
        return  "Russia"
    elif col == "Montreal":
        return  "Canada"
    elif col == "Munich" or col == "Berlin":
        return  "Germany"
    elif col == "Mexico City":
        return  "Mexico"
    elif col == "Tokyo":
        return  "Japan"
    elif col == "Roma":
        return  "Italy"
    elif col == "Paris":
        return  "France"
    elif col == "Helsinki":
        return  "Finland"
    elif col == "Amsterdam":
        return  "Netherlands"
    elif col == "Antwerpen":
        return  "Belgium"
    elif col == "Stockholm":
        return  "Sweden"
    else:
        return "Other"


# Applying this function

df['Host_Country'] = df['City'].apply(host_country)

In [ ]:
df_new = df.groupby(['Year','Host_Country','region','Medal'])['Medal'].count().unstack().fillna(0).astype(int).reset_index()

df_new['Is_Host'] = np.where(df_new['Host_Country'] == df_new['region'],1,0)
df_new['Total Medals'] = df_new['Bronze'] + df_new['Silver'] + df_new['Gold']

In [ ]:
# Preparing DF 2021 Dataset

df_21_refined = df_21[['Team/NOC', "Gold Medal", "Silver Medal", "Bronze Medal"]]
df_21_refined['Total Medals'] = df_21_refined[["Gold Medal", "Silver Medal", "Bronze Medal"]].sum(axis=1)
df_21_refined['Year'] = 2021

df_21_refined = df_21_refined.rename(columns={'Gold Medal':'Gold', 'Silver Medal':'Silver','Bronze Medal':'Bronze'})

df_21_refined['Is_Host'] = np.where(df_21_refined['Team/NOC'] == 'Japan',1,0)
df_21_refined['Host_Country'] = 'Japan'
df_21_refined = df_21_refined.rename(columns={'Team/NOC':'region'})

# Adding 2021 data to historic
df_full = pd.concat([df_new, df_21_refined])

In [ ]:
df_full

In [ ]:
china_data = {'Year':['2022','2018', '2014', '2010', '2006', '2002'],'Total Medals':[61, 1, 0, 0, 0, 0]}

df_china_paralympics = pd.DataFrame(china_data)

In [ ]:
background_color = '#ECE7E2'
primary_color = '#2C4545'
secondary_color = '#B57B68'

### China in paralympics

Source: https://en.wikipedia.org/wiki/China_at_the_Paralympics#Medals_by_Winter_Games

China first participated in the Winter Olympics in 2002, won their first medal in 2018 and won 61 more in 2022 when they hosted the games.

In [ ]:
data = [
    go.Pie(labels=df_china_paralympics['Year'][:2],
           values=df_china_paralympics['Total Medals'][:2],
           marker_colors=[primary_color, secondary_color])
]

layout = go.Layout(
    title_text = 'Distribution of the Medals won of China over the years',
    title_font_size = 25,
    width = 800,
    height = 550,
    paper_bgcolor = background_color
)

fig1 = go.Figure(data, layout)

fig1.update_xaxes(title = 'Medal-Distribution')

fig1.show(config={'displayModeBar': False})

As you can see, they won over 98% of all their medals in this years Winter Paralympics. While we can speculate if their athletes just worked that hard or if something else is the reason here, it got me interested if hosting the olympics had a influence of the medals as well. In the following Visualizations I limited the data to just the summer olympics, as it seems more popular across the whole earth and all continents.

### 2021 Olympics

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
df_fig2 = df_full[df_full['Year'] == 2021]
df_fig2 = df_fig2.sort_values(by = ['Total Medals'], ascending = False).iloc[0:10].reset_index()
df_fig2['index_column'] = df_fig2.index

In [ ]:
# Draw points
data = [
    go.Scatter(
        name='',
        x=df_fig2['Total Medals'],
        y=df_fig2['region'],
        mode='markers',
        marker=dict(color=primary_color, size=15)
    ),
    
    go.Scatter(
        name='',
        x=[df_fig2['Total Medals'][4]],
        y=[df_fig2['region'][4]],
        mode='markers',
        marker=dict(color=secondary_color, size=25)
    )
]

layout = go.Layout(
    title_text = 'Total Medals 2021 - Top 10 Countries',
    title_font_size = 30,
    width = 800,
    height = 550,
    plot_bgcolor = background_color
)

fig2 = go.Figure(data, layout)

# Draw lines
for i in range(0, len(df_fig2)):
    if df_fig2['Is_Host'][i] == 0:
        fig2.add_shape(type='line',
                       x0 = 0, y0 = i,
                       x1 = df_fig2['Total Medals'][i],
                       y1 = i,
                       line=dict(color=primary_color, width = 1))
    else:
        fig2.add_shape(type='line',
                   x0 = 0, y0 = i,
                   x1 = df_fig2['Total Medals'][i],
                   y1 = i,
                   line=dict(color=secondary_color, width = 1))

        
anno1 = {
    'x': 62, # end of the arrow
    'y': 4.3,
    'xref': 'x', 
    'yref': 'y', 
    'text': 'In 2021,<br />Japan got 58 Medals,<br />which saw them ending<br />the olympics in 5. Place',
    'font': {'size': 12, 'color': primary_color}, 
    'arrowhead': 2, 
    'arrowsize': 1, 
    'arrowwidth': 2, 
    'arrowcolor': primary_color,
    'ax': 100, # start of the arrow / text position  
    'ay': 8, 
    'axref': 'x', 
    'ayref': 'y'
}
fig2.add_annotation(anno1)

anno2 = {
    'text': '58',
    'font': {'size': 12, 'color': 'white'}, 
    'ax': 58,
    'ay': 4, 
    'x': 58,
    'y': 4, 
    'axref': 'x', 
    'ayref': 'y',
}
fig2.add_annotation(anno2)

fig2.update(layout_showlegend=False)
fig2.update_xaxes(title = 'Total Medals', range=[0, max(df_fig2['Total Medals']) + 5], showgrid=False, fixedrange = True)
fig2.update_yaxes(showgrid=False, fixedrange = True)

fig2.show(config={'displayModeBar': False})

I wanted to take red as the highlighting color, because its good visible and the red dot fits to the japanese flag, but I found out that you cant hardly differentiate red and the dark-blue with monochromatic vision [1] , which around 1 in 30'000 People have. Which this lighter brown-red color, the contrast gets better to the dark blue and it should be visible with all kind of color-blindness.

[1]=https://en.wikipedia.org/wiki/Monochromacy

### Comparing to non-Hosting Years

But how does this compare to the years, when a country isn't hosting the olympics. For that, we take the Top 15 Nations of all time and calculate their average total Medal count and the average when hosting the olympics.

In [ ]:
top15_df = df_new.groupby('region')['Total Medals'].mean().sort_values(ascending=False).reset_index()[:15].sort_values(by='Total Medals',ascending=True)

In [ ]:
# Draw mean points
layout = go.Layout(
    title_text = 'Medals over the Years for Top 15 Countries,<br />with Average and Hosting Years highlighted',
    title_font_size = 20,
    title_xanchor = 'center',
    title_x = 0.5,
    width = 1000,
    height = 600,
    plot_bgcolor = background_color,
)

fig3 = go.Figure(layout=layout)

for country in top15_df['region']:
    
    # Add history-data
    hosting_medals = df_new[(df_new['region'] == country)]['Total Medals']
    country_array = [country] * hosting_medals.size
    fig3.add_trace(
        go.Scatter(
            name='',
            x=country_array,
            y=hosting_medals,
            mode='markers',
            marker=dict(color='lightgray', size=7)
        )
    )
    
    # Add hostingpoints
    hosting_medals = df_new[(df_new['region'] == country) & (df_new['Is_Host'] == 1)]['Total Medals']
    country_array = [country] * hosting_medals.size
    fig3.add_trace(
        go.Scatter(
            name='',
            x=country_array,
            y=hosting_medals,
            mode='markers',
            marker=dict(color=secondary_color, size=12),
            hovertemplate= '%{y} Medals when hosting'
        )
    )

# Add Average Points
fig3.add_trace(
    go.Scatter(
        name='',
        x=top15_df['region'],
        y=top15_df['Total Medals'],
        mode='markers',
        marker=dict(color=primary_color, size=12),
        hovertemplate= '%{y} Medals on Average'
    )
)

anno1 = {
    'x': 3.9, # end of the arrow
    'y': 55,
    'xref': 'x', 
    'yref': 'y', 
    'text': 'Average Medals',
    'font': {'size': 15, 'color': primary_color}, 
    'arrowhead': 2, 
    'arrowsize': 1, 
    'arrowwidth': 2, 
    'arrowcolor': primary_color,
    'ax': 2, # start of the arrow / text position  
    'ay': 300, 
    'axref': 'x', 
    'ayref': 'y'
}

anno2 = {
    'x': 4.1, # end of the arrow
    'y': 205,
    'xref': 'x', 
    'yref': 'y', 
    'text': 'Medals when hosting the Olympics',
    'font': {'size': 15, 'color': secondary_color}, 
    'arrowhead': 2, 
    'arrowsize': 1, 
    'arrowwidth': 2, 
    'arrowcolor': secondary_color,
    'ax': 6, # start of the arrow / text position  
    'ay': 450, 
    'axref': 'x', 
    'ayref': 'y'
}

fig3.add_annotation(anno1)
fig3.add_annotation(anno2)

fig3.update(layout_showlegend=False)
fig3.update_xaxes(title = 'Comparing Average Medals with Medals when Hosting', showgrid=True, gridcolor='lightgray', fixedrange = True)
fig3.update_yaxes(showgrid=False, fixedrange = True, range=[0, 500])

fig3.show(config={'displayModeBar': False})

As can be seen here, with some few exceptions like Sweden once and the United Kingdom once, the hosting Years seem to be mostly clearly higher than the average.
As we can see here as well, is that on average, Japan is only on 10th Position, in contrast to their 2021 campaign, where they ended the olympics in 5th Place.